In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.datasets import load_digits
from sklearn.model_selection import validation_curve
import time
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

In [ ]:
!pip install torch torchvision
!pip install torch-geometric torch-sparse torch-scatter torch-cluster

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
pre_df = pd.read_csv('cleaned_vessel_calls.csv').sample(n=6000)

In [ ]:
pre_df.head()

In [ ]:
# preprocess dataframe - duration and month
pre_df["arrivalDate"] = pd.to_datetime(pre_df["arrivalDate"])
pre_df["sailingDate"] = pd.to_datetime(pre_df["sailingDate"])
pre_df["duration"] = pd.to_datetime(pre_df["sailingDate"]) - pd.to_datetime(pre_df["arrivalDate"])
pre_df["month"] = pre_df["arrivalDate"].dt.month

In [ ]:
pre_df['duration'] = pre_df['duration'] / pd.Timedelta(days=1)

In [ ]:
# drop null values in df
pre_df.dropna(inplace=True)

In [ ]:
# Encode the target variable (port name)
le_target = LabelEncoder()

pre_df['cur_port_encoded'] = le_target.fit_transform(pre_df['place.unlocode'].astype(str))

In [ ]:
# preprocess data
# sort by imo number
pre_df.sort_values(by=["IMO", "arrivalDate"], inplace=True)
pre_df['next_place'] = pre_df.groupby('IMO')['place.unlocode'].shift(-1)
pre_df['next_place_encoded'] = pre_df.groupby('IMO')['cur_port_encoded'].shift(-1)
pre_df['is_last_imo'] = pre_df['IMO'] != pre_df['IMO'].shift(-1)
pre_df = pre_df[~pre_df["is_last_imo"]]
pre_df = pre_df[pre_df["Event_Type"] != "ARRIVAL"]

In [ ]:
pre_df = pre_df.drop(columns=["is_last_imo", "Event_Type", "Timestamp", "vessel.vesselName", "IMO", "arrivalDate", "sailingDate"])
pre_df

,place.placeName,place.unlocode,place.placeType,place.countryName,vessel.vesselType,duration,month,cur_port_encoded,next_place,next_place_encoded
135911,Kakogawa,JPKGA,Sub Port,Japan,tug/supply,0.333345,6,361,JPTHS,382.0
135957,Toyohashi,JPTHS,Port,Japan,tug/supply,1.329815,11,382,PETYL,521.0
136047,Talara,PETYL,Port,Peru,tug/supply,0.410150,3,521,PETYL,521.0
136101,Talara,PETYL,Port,Peru,tug/supply,1.251366,5,521,PETYL,521.0
136137,Talara,PETYL,Port,Peru,tug/supply,0.485903,5,521,PETYL,521.0
...,...,...,...,...,...,...,...,...,...,...
100401,Port Klang (Pelabuhan Klang),MYPKG,Port,Malaysia,patrol ship,0.625150,8,459,CNSHK,166.0
100427,Shekou Port,CNSHK,Sub Port,People's Republic of China,patrol ship,0.749954,9,166,KRBNP,392.0
100479,Busan New Port,KRBNP,Sub Port,Republic of Korea,patrol ship,0.986100,12,392,INHZA,308.0
100543,Shanghai,CNSGH,Port,People's Republic of China,patrol ship,0.571516,4,165,BRSSZ,91.0


In [ ]:
pre_df[['place.unlocode', 'next_place', 'cur_port_encoded', 'next_place_encoded']].head(1000)

In [ ]:
pre_df['next_place_encoded'] = pre_df['next_place_encoded'].astype(int)

In [ ]:
pre_df = pre_df.drop(columns=["place.placeName"])
df = pre_df.rename(columns={'place.unlocode':'unlocode','vessel.vesselType':'vesselType', 'place.placeType': 'placeType', 'place.countryName': 'countryName'})
df

In [ ]:
# data statistics
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2499 entries, 135911 to 100563
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   unlocode            2499 non-null   object 
 1   placeType           2499 non-null   object 
 2   countryName         2499 non-null   object 
 3   vesselType          2499 non-null   object 
 4   duration            2499 non-null   float64
 5   month               2499 non-null   int32  
 6   cur_port_encoded    2499 non-null   int64  
 7   next_place          2499 non-null   object 
 8   next_place_encoded  2499 non-null   int64  
dtypes: float64(1), int32(1), int64(2), object(5)
memory usage: 185.5+ KB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,2499.0,1.497790,3.339602,0.005521,0.581898,0.999225,1.675839,122.414248
month,2499.0,5.943177,3.417794,1.000000,3.000000,6.000000,9.000000,12.000000
cur_port_encoded,2499.0,391.139256,210.001748,2.000000,178.000000,399.000000,593.000000,740.000000
next_place_encoded,2499.0,396.397759,210.863299,1.000000,179.500000,400.000000,593.000000,740.000000


In [ ]:
cap = df['duration'].quantile(.99)
df['capped_duration'] = df['duration'].apply(lambda x: min(x, cap))

In [ ]:
# convert next place encoded to an int
df['next_place_encoded'] = df['next_place_encoded'].astype(int)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,2499.0,1.497790,3.339602,0.005521,0.581898,0.999225,1.675839,122.414248
month,2499.0,5.943177,3.417794,1.000000,3.000000,6.000000,9.000000,12.000000
cur_port_encoded,2499.0,391.139256,210.001748,2.000000,178.000000,399.000000,593.000000,740.000000
next_place_encoded,2499.0,396.397759,210.863299,1.000000,179.500000,400.000000,593.000000,740.000000
capped_duration,2499.0,1.357875,1.288854,0.005521,0.581898,0.999225,1.675839,8.706633


In [ ]:
# Encode categorical features
label_encoders = {}
for column in ['placeType', 'countryName', 'vesselType']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))
    label_encoders[column] = le

In [ ]:
df

In [ ]:
# do the shifting after the encoding.
df = df.drop(columns=["unlocode", "countryName", "next_place", "duration"])
df

,placeType,vesselType,month,cur_port_encoded,next_place_encoded,capped_duration
135911,2,14,6,361,382,0.333345
135957,1,14,11,382,521,1.329815
136047,1,14,3,521,521,0.410150
136101,1,14,5,521,521,1.251366
136137,1,14,5,521,521,0.485903
...,...,...,...,...,...,...
100401,1,11,8,459,166,0.625150
100427,2,11,9,166,392,0.749954
100479,2,11,12,392,308,0.986100
100543,1,11,4,165,91,0.571516


In [ ]:
port_ids = df['cur_port_encoded']
next_port_ids = df['next_place_encoded']

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
Tesla T4


In [ ]:
from torch_geometric.data import Data

# Node features (could be more sophisticated in real use)
node_features = torch.eye(len(le_target.classes_))  # Simple one-hot encoding for demonstration

# Edge index (source to target)
edge_index = torch.tensor([port_ids[:-1].to_numpy(), next_port_ids[1:].to_numpy()], dtype=torch.long)

# Edge features (could include normalized duration, month as cyclic encoding, etc.)
edge_features = torch.tensor([
    df['capped_duration'].values[:-1],  # simple direct use, should be normalized
    np.cos(df['month'].values[:-1] * 2 * np.pi / 12),  # cyclic encoding of month
    np.sin(df['month'].values[:-1] * 2 * np.pi / 12)
], dtype=torch.float).t()

# Define graph data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features).to(torch.device('cuda'))

In [ ]:
print("Number of edges:", edge_index.shape[1])
print("Number of edge weights:", data.edge_attr.shape[0])

Number of edges: 2498
Number of edge weights: 2498


In [ ]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import NNConv
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
import torch.nn as nn

# Model Definition
class Net(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, num_classes):
        super(Net, self).__init__()
        nn = Sequential(Linear(num_edge_features, 20), ReLU(), Linear(20, num_node_features * 20))
        self.conv1 = NNConv(num_node_features, 20, nn, aggr='mean')
        self.conv2 = NNConv(20, num_classes, nn, aggr='mean')

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)

model = Net(num_node_features=node_features.size(1), num_edge_features=edge_features.size(1), num_classes=len(le_target.classes_)).to(torch.device('cuda'))

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
# Training Configuration
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training and Evaluation Functions
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)  # Define data.y appropriately based on your data
    loss.backward()
    optimizer.step()
    return loss

def evaluate():
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = int(pred.eq(data.y).sum().item())
    acc = correct / data.num_nodes
    return acc

In [ ]:
# Training Loop with Learning Curve Generation
losses, accuracies = [], []
for epoch in range(200):
    loss = train()
    acc = evaluate()
    losses.append(loss.item())
    accuracies.append(acc)
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}, Accuracy: {acc}')

# Plotting Learning and Validation Curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(losses, label='Training Loss')
plt.title('Learning Curve - Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracies, label='Validation Accuracy')
plt.title('Learning Curve - Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not NoneType